# Section 1-3 - Parameter Tuning

In previous sections, we took the approach of using Scikit-learn as a black box. We now review how to tune the parameters of the model to make more accurate predictions.

## Pandas - Extracting data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')

df_train = df.iloc[:712, :]
df_test = df.iloc[712:, :]

## Pandas - Cleaning data

In [2]:
df_train = df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)

df_train['Embarked'] = df_train['Embarked'].fillna('S')

df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)

df_train = df_train.drop(['Embarked'], axis=1)

X_train = df_train.iloc[:, 2:].values
y_train = df_train['Survived']

## Scikit-learn - Training the model

The documentation for the Random Forest Classifier details the different input parameters of the model. These input parameters include the number of trees, and the number of branches each tree has. It is unclear, off-the-bat, which values would be optimal.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

GridSearchCV allows us to test the desired range of input parameters, and review the performance of each set of values on a cross-validation basis. Here we review the number of features considered at each step a branch is made (max_features: 50% or 100% of features) and the maximum number of branches (max_depth: 5 levels or no limitations).

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameter_grid = {
    'max_features': [0.5, 1.],
    'max_depth': [5., None]
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid, cv=5, verbose=3)

In [4]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...max_depth=5.0, max_features=0.5;, score=0.790 total time=   0.1s
[CV 2/5] END ...max_depth=5.0, max_features=0.5;, score=0.783 total time=   0.1s
[CV 3/5] END ...max_depth=5.0, max_features=0.5;, score=0.901 total time=   0.1s
[CV 4/5] END ...max_depth=5.0, max_features=0.5;, score=0.831 total time=   0.1s
[CV 5/5] END ...max_depth=5.0, max_features=0.5;, score=0.775 total time=   0.1s
[CV 1/5] END ...max_depth=5.0, max_features=1.0;, score=0.811 total time=   0.1s
[CV 2/5] END ...max_depth=5.0, max_features=1.0;, score=0.804 total time=   0.1s
[CV 3/5] END ...max_depth=5.0, max_features=1.0;, score=0.887 total time=   0.1s
[CV 4/5] END ...max_depth=5.0, max_features=1.0;, score=0.817 total time=   0.1s
[CV 5/5] END ...max_depth=5.0, max_features=1.0;, score=0.782 total time=   0.1s
[CV 1/5] END ..max_depth=None, max_features=0.5;, score=0.776 total time=   0.1s
[CV 2/5] END ..max_depth=None, max_features=0.5;,

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5.0, None], 'max_features': [0.5, 1.0]},
             verbose=3)

We now review the results.

In [5]:
grid_search.cv_results_['mean_test_score']

array([0.81609377, 0.82026002, 0.80344726, 0.80480646])

We now review the best-performing tuning parameters.

In [6]:
grid_search.best_params_

{'max_depth': 5.0, 'max_features': 1.0}

We then set these tuning parameters to our model.

In [7]:
model = RandomForestClassifier(n_estimators = 100, max_features=1.0, max_depth=5.0, random_state=0)
model = model.fit(X_train, y_train)

**Exercise**

- Write the code so that grid_search refits model with the best tuning parameters to the entire dataset after these parameters are found, and hence allow us to skip the two lines of code above.

## Scikit-learn - Making predictions

In [8]:
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Embarked'] = df_test['Embarked'].fillna('S')

df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],axis=1)

df_test = df_test.drop(['Embarked'], axis=1)

X_test = df_test.iloc[:, 2:]
y_test = df_test['Survived']

y_prediction = model.predict(X_test)

## Evaluation

In [9]:
np.sum(y_prediction == y_test) / float(len(y_test))

0.8603351955307262